# Проект по подсчету статистики по выпускаемым актам

## Описание

Проект направлен на автоматизацию процесса сбора статистики по количеству выпущенных актов. Пользователю необходимо создавать отчеты раз в месяц, и программа упрощает этот процесс, собирая необходимые данные и формируя отчет в формате .xlsx в виде таблиц с графической частью.

## User Story

Требуется автоматическая генерация отчета по количеству выпущенных актов выявленных дефектов оборудования раз в месяц. В настоящее время процедура сбора данных и составления отчета занимает от 3 до 5 дней.

## Результаты предварительного анализа документации пользователя

Пользователь создает табличные отчеты в Excel, при этом наблюдаются ошибки при заполнении. Типичные примеры ошибок включают неверный формат даты. Пользователь не имеет системы учета, что препятствует дублированию нумерации актов. Проведен сравнительный анализ количества выпускаемых актов, и принято решение использовать данные о дате последнего изменения файла в качестве основы для построения отчетности.

## Задача

Написать программу для автоматического сбора требуемых данных. Конечный отчет должен быть представлен в формате .xlsx и содержать графическую часть для наглядного отображения информации.

## Используемый стек технологий

- Python: Основной язык программирования для разработки.
- Pandas: Библиотека для обработки и анализа данных.
- OpenPyXL: Библиотека для работы с файлами Excel (.xlsx).
- OS: Модуль для взаимодействия с операционной системой (например, для работы с файловой системой).
- MS Excel: Программа для просмотра и редактирования созданных отчетов.
- Random: Библиотека для генерации случайных чисел и выбора случайных элементов.
- Datetime: Модуль для работы с датами и временем, позволяющий легко получать текущую дату и время, а также выполнять операции с ними.

## Импорт библиотек

In [6]:
import pandas as pd
import openpyxl as op
from openpyxl.chart import BarChart, Reference
import os
import random
from datetime import datetime, timedelta

## Код для генерации искуственных файлов

In [7]:
#КОД ДЛЯ ГЕНЕРАЦИИ ИСКУСТВЕННЫХ ДАННЫХ
# директория для генерации искуственных данных
output_directory = r'C:\Users\Admin\Desktop\генерация_актов'

#ЗАДАЕМ СЛУЧАЙНОЕ ЧИСЛО ФАЙЛОВ
# Генерируем случайное количество файлов от 1 до 300
num_files = random.randint(1, 300)

# Функция для генерации случайной даты
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

#ЗАДАЕМ ДИАПАЗОН ДАТ
# Задаем диапазон дат
start_date = datetime(2020, 1, 1)
end_date = datetime.now()

#СОЗДАЕМ ФАЙЛЫ
# Создаем файлы
for i in range(1, num_files + 1):
    file_name = f'акт_{i}.xlsx'  # Название файла
    file_path = os.path.join(output_directory, file_name)  # Полный путь к файлу
    with open(file_path, 'w') as f:
        pass  # Создаем пустой файл
#ГЕНЕРАЦИЯ ДАТЫ ПОСЛЕДНЕГО ИЗМЕНЕНИЯ    
    # Генерируем случайную дату последнего изменения
    last_modified_date = random_date(start_date, end_date)       
    # Изменяем дату последнего изменения файла
    os.utime(file_path, (last_modified_date.timestamp(), last_modified_date.timestamp()))

print(f'Создано {num_files} пустых файлов с случайными датами последнего изменения в директории: {output_directory}')


Создано 51 пустых файлов с случайными датами последнего изменения в директории: C:\Users\Admin\Desktop\генерация_актов


## Код проекта

In [8]:
#КОД ПРОЕКТА
#ЗАГРУЗКА ФАЙЛА
# Путь к папке с файлами excel
folder_path = output_directory  # заменить путь на папку с файлами при необходимости

# Список для хранения информации о файлах
file_info = []

## Перебираем все файлы в указанной папке и подпапках
for dirpath, dirnames, filenames in os.walk(folder_path):
    for filename in filenames:
        if filename.endswith('.xlsx') or filename.endswith('.xls'):  # Проверка ,что файл действительно .xlsx или .xls
            file_path = os.path.join(dirpath, filename)
            last_modified_time = os.path.getmtime(file_path)  # Получаем время последнего изменения
            last_modified_date = pd.to_datetime(last_modified_time, unit='s')  # Преобразуем в дату
            year = last_modified_date.year
            month = last_modified_date.month
            file_info.append({'Имя файла': filename, 'Дата последнего изменения': last_modified_date, 'Год': year, 'Месяц': month})

#УСЛОВИЕ ПРОВЕРКИ НА НАЛИЧИЕ EXCEL ФАЙЛОВ В ПАПКАХ
# Условие для проверки на наличие файлов
if not file_info:
    print("Нет файлов для обработки.")
else:
    # Создаем датафрейм из списка file_info
    df = pd.DataFrame(file_info)

    # Считаем количество актов посредством группировки по месяцам
    monthly_counts = df.groupby(['Год', 'Месяц']).size().reset_index(name='Количество')

    # Объединяем год и месяц в одну строку для оси X
    monthly_counts['Год.Месяц'] = monthly_counts['Год'].astype(str) + '.' + monthly_counts['Месяц'].apply(lambda x: f'{x:02}')

    # Генерируем полный список месяцев и годов
    all_months = pd.MultiIndex.from_product([df['Год'].unique(), range(1, 13)], names=['Год', 'Месяц'])
    monthly_counts = monthly_counts.set_index(['Год', 'Месяц']).reindex(all_months, fill_value=0).reset_index()

    # Фильтруем только те строки, где количество актов больше нуля
    filtered_counts = monthly_counts[monthly_counts['Количество'] > 0]

    # Выводим датафреймы раскомитить при необходимости
    #print("Информация о файлах:")
    #print(df)
    #print("\\nКоличество актов по месяцам (с учетом только положительных значений):")
   # print(filtered_counts)

#ВЫВОД ФАЙЛА
    # Сохраняем датафреймы в Excel файл
    output_file = os.path.join(output_directory, 'отчет.xlsx')

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, sheet_name='Информация об Актах', index=False)
        monthly_counts.to_excel(writer, sheet_name='Количество Актов по месяцам', index=False)
        
#ДОБАВЛЕНИЕ ГРАФИКА
    # Открываем созданный файл для добавления графика
    wb = op.load_workbook(output_file)

    # Автоматическое выравнивание ширины колонок для листа "Информация об Актах"
    sheet_info = wb['Информация об Актах']
    for column in sheet_info.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)  # Добавляем небольшой отступ
        sheet_info.column_dimensions[column[0].column_letter].width = adjusted_width

#НАСТРОЙКИ ШИРИНЫ КОЛОНОК ЛИСТОВ
    # Автоматическое выравнивание ширины колонок для листа "Количество Актов по месяцам"
    sheet_counts = wb['Количество Актов по месяцам']
    for column in sheet_counts.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)  # Добавляем небольшой отступ
        sheet_counts.column_dimensions[column[0].column_letter].width = adjusted_width

#НАСТРОЙКИ ГРАФИКА
    # Создаем новый лист для добавления графика
    chart_sheet = wb.create_sheet(title='График')

#НАСТРОЙКИ ФИЛЬТРА ДЛЯ ОСИ X
    # Сортируем filtered_counts по Году и Месяцу
    filtered_counts = filtered_counts.sort_values(by=['Год', 'Месяц'])

#ДОБАВЛЕНИЕ ЗАГОЛОВКОВ ДЛЯ ГРАФИКА
    # Добавляем заголовки для графика
    chart_sheet.append(['Год', 'Месяц', 'Количество', 'Год.Месяц'])

#Заполняем новый лист данными для графика из отфильтрованного датафрейма
    for row in filtered_counts.itertuples(index=False):
        chart_sheet.append(row)

#НАСТРОЙКИ ШИРИНЫ КОЛОНОК ЛИСТА ГРАФИКА        
# Автоматическое выравнивание ширины колонок для листа "График"
    sheet_counts = wb['График']
    for column in sheet_counts.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)  # Добавляем небольшой отступ
        sheet_counts.column_dimensions[column[0].column_letter].width = adjusted_width

#ГРАФИК   
    # Создание графика
    chart = BarChart()
    data = Reference(chart_sheet, min_col=3, min_row=1, max_col=3, max_row=len(filtered_counts)+1)  # Количество актов
    categories = Reference(chart_sheet, min_col=4, min_row=2, max_row=len(filtered_counts)+1)  # Годы и месяцы

    chart.add_data(data, titles_from_data=True)
    chart.set_categories(categories)
    chart.title = "Количество актов по месяцам"
    chart.x_axis.title = "Год и месяц"
    chart.y_axis.title = "Количество актов"

#ГРАФИК  МЕТКИ ДАННЫХ
    # Добавление меток данных на график
    for series in chart.series:
        series.dLbls = op.chart.label.DataLabelList()
        series.dLbls.showVal = True  # Показывать значения

#ДОБАВЛЕНИЕ ГРАФИКА НА ЛИСТ И ПОЗИЦИОНИРОВАНИЕ
    # Добавление графика на лист
    chart_sheet.add_chart(chart, "E5")  # Позиция, где будет размещен график

#ВЫВОД ОТЧЕТА
    # Сохранение файла с графиком и выравниванием
    wb.save(output_file)

    print(f"Данные сохранены в {output_file} с графиком на листе 'График'.")


Данные сохранены в C:\Users\Admin\Desktop\генерация_актов\отчет.xlsx с графиком на листе 'График'.
